In [20]:
# target parameter attack on linear regression with close form solution on the cross derivative

import os
import time
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from numpy import linalg as LA
import numpy as np
import math
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib
from sklearn.datasets import make_classification


torch.manual_seed(0)
device = 'cuda'

In [149]:
# creating the gaussian dataset

# define training set
separable = False
while not separable:
    samples = make_classification(n_samples=1000, n_features=3, n_redundant=0, n_informative=1, n_clusters_per_class=1, flip_y=-1)
    red = samples[0][samples[1] == 0]
    blue = samples[0][samples[1] == 1]
    separable = any([red[:, k].max() < blue[:, k].min() or red[:, k].min() > blue[:, k].max() for k in range(2)])
red_labels = np.zeros(len(red))
blue_labels = np.ones(len(blue))

labels = np.append(red_labels,blue_labels)
inputs = np.concatenate((red,blue),axis=0)

X_train, X_test, y_train,  y_test = train_test_split(
    inputs, labels, test_size=0.33, random_state=42)

X_train, X_test = torch.Tensor(X_train),torch.Tensor(X_test)
y_train, y_test = torch.Tensor(y_train),torch.Tensor(y_test)



In [150]:
batch_size_train= len(X_train)
batch_size_test = len(X_test)
class LinearDataset(Dataset):
    def __init__(self, X, y):
        assert X.size()[0] == y.size()[0]
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size()[0]

    def __getitem__(self, idx):
        
        
        return [self.X[idx], self.y[idx]]
train_loader = DataLoader(LinearDataset(X_train, y_train), batch_size=batch_size_train, shuffle=False)
test_loader = DataLoader(LinearDataset(X_test, y_test), batch_size=batch_size_test, shuffle=False)

In [151]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

input_dim = 3 
output_dim = 1 # Two possible outputs
learning_rate = 0.01

model = LogisticRegression(input_dim,output_dim).to(device)
model.load_state_dict(torch.load("gaussian_gd_0.5.pt"))

<All keys matched successfully>

In [152]:
from scipy.special import lambertw
import math
lambert_w = lambertw(1/math.e)
print(lambert_w)

(0.2784645427610738+0j)


In [153]:
for param in model.parameters():
    w_p = param
    break
print(w_p/10)

tensor([[-0.0346, -0.2471,  0.0194]], device='cuda:0', grad_fn=<DivBackward0>)


In [154]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linear.weight tensor([[-0.3458, -2.4712,  0.1935]], device='cuda:0')
linear.bias tensor([1.2288], device='cuda:0')


In [155]:
# try scaling the weights 
state_dict = model.state_dict()
state_dict['linear.weight'] = torch.tensor([[-0.03458,-0.24712,0.01935]])
model.load_state_dict(state_dict)

<All keys matched successfully>

In [158]:
# the script for training target attack
epsilon = 1
lr = 0.005
epochs = 3000

def adjust_learning_rate(lr, epoch):
    """Decay the learning rate based on schedule"""
    lr *= 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    return(lr)


def autograd(outputs, inputs, create_graph=False):
    """Compute gradient of outputs w.r.t. inputs, assuming outputs is a scalar."""
    #inputs = tuple(inputs)
    grads = torch.autograd.grad(outputs, inputs, create_graph=create_graph, allow_unused=True)
    return [xx if xx is not None else yy.new_zeros(yy.size()) for xx, yy in zip(grads, inputs)]

def train(epoch, X,y):
    data, target = X.to(device), y.to(device)
    data.requires_grad=True
    if epoch==0:
        # initialize poisoned data
        data_p = Variable(data[:(int(epsilon*len(data)))])
        target_p = Variable(target[:(int(epsilon*len(target)))])
        torch.save(target_p,'target_p_gaussian_{}.pt'.format(epsilon))
    else:
        data_p = torch.load('data_p_gaussian_{}.pt'.format(epsilon))
        target_p = torch.load('target_p_gaussian_{}.pt'.format(epsilon))
    data_p.requires_grad=True

    # initialize f function
    criterion = torch.nn.BCELoss(reduction='sum')

    # calculate gradient of w on clean sample
    output_c = torch.squeeze(model(data))
    loss_c =  0.5 * criterion(output_c,target)

    # calculate dL/dg_1
    grad_c= autograd(loss_c,tuple(model.parameters()),create_graph=True)
    g1 = grad_c[0]/len(y)

    # calculate the size of epsilon_d w.r.t g1 and the Lambert's W function
    g_mu_dot_w = np.dot(g1.to('cpu').detach().numpy().squeeze(),w_p.cpu().detach().numpy().squeeze())
    if epoch == 0:
        print(g_mu_dot_w)
        print('the necessary size of epsilon_d:{}'.format(g_mu_dot_w/lambert_w))
    

    # calculate gradient of w on poisoned sample
    output_p = torch.squeeze(model(data_p))
    loss_p = 0.5 * criterion(output_p,target_p)
    grad_p= autograd(loss_p,tuple(model.parameters()),create_graph=True)

    g2 = torch.matmul((output_p - target_p).t(),data_p)
    # calculate the true loss: |g_c + g_p|_{2}
    grad_sum = g1+g2
    loss = torch.norm(grad_sum,2)

    update = autograd(loss,data_p,create_graph=True)
    data_t = data_p - lr * update[0]


    data_t = data_t.to('cuda')

    torch.save(data_t, 'data_p_gaussian_{}.pt'.format(epsilon))
    

    print("epoch:{},lr:{},loss:{}".format(epoch,lr,loss))
    
    return loss
        

In [159]:
for epoch in range(epochs):
    loss = train(epoch,X_train,y_train)
    if loss<0.0001:
        break

0.0679838
the necessary size of epsilon_d:(0.24413807951492283+0j)
[tensor([[-0.0151,  0.3248, -0.0140],
        [ 0.0599, -0.8156,  0.0309],
        [-0.0153,  0.3253, -0.0139],
        ...,
        [-0.0143,  0.3290, -0.0143],
        [ 0.0616, -0.8409,  0.0318],
        [ 0.0626, -0.8498,  0.0321]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:0,lr:0.005,loss:373.5389404296875
[tensor([[-0.0151,  0.3247, -0.0139],
        [ 0.0599, -0.8152,  0.0308],
        [-0.0154,  0.3251, -0.0139],
        ...,
        [-0.0143,  0.3288, -0.0143],
        [ 0.0617, -0.8406,  0.0318],
        [ 0.0626, -0.8495,  0.0321]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:1,lr:0.005,loss:372.1507568359375
[tensor([[-0.0151,  0.3245, -0.0139],
        [ 0.0599, -0.8149,  0.0308],
        [-0.0154,  0.3249, -0.0139],
        ...,
        [-0.0144,  0.3287, -0.0143],
        [ 0.0617, -0.8403,  0.0318],
        [ 0.0627, -0.8492,  0.0321]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2,lr:0.005

epoch:34,lr:0.005,loss:326.9038391113281
[tensor([[-0.0157,  0.3190, -0.0135],
        [ 0.0613, -0.8044,  0.0300],
        [-0.0159,  0.3194, -0.0135],
        ...,
        [-0.0149,  0.3231, -0.0139],
        [ 0.0631, -0.8304,  0.0310],
        [ 0.0640, -0.8395,  0.0313]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:35,lr:0.005,loss:325.5498352050781
[tensor([[-0.0157,  0.3188, -0.0135],
        [ 0.0613, -0.8041,  0.0300],
        [-0.0160,  0.3192, -0.0135],
        ...,
        [-0.0149,  0.3229, -0.0139],
        [ 0.0631, -0.8301,  0.0310],
        [ 0.0641, -0.8392,  0.0313]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:36,lr:0.005,loss:324.1968078613281
[tensor([[-0.0157,  0.3186, -0.0135],
        [ 0.0613, -0.8037,  0.0300],
        [-0.0160,  0.3190, -0.0135],
        ...,
        [-0.0150,  0.3228, -0.0139],
        [ 0.0631, -0.8298,  0.0310],
        [ 0.0641, -0.8389,  0.0313]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:37,lr:0.005,loss:322.8447875976562

[tensor([[-0.0164,  0.3123, -0.0131],
        [ 0.0629, -0.7911,  0.0290],
        [-0.0166,  0.3127, -0.0130],
        ...,
        [-0.0156,  0.3163, -0.0134],
        [ 0.0648, -0.8177,  0.0301],
        [ 0.0657, -0.8272,  0.0304]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:76,lr:0.005,loss:270.9007873535156
[tensor([[-0.0164,  0.3121, -0.0131],
        [ 0.0630, -0.7907,  0.0290],
        [-0.0167,  0.3125, -0.0130],
        ...,
        [-0.0156,  0.3162, -0.0134],
        [ 0.0648, -0.8174,  0.0300],
        [ 0.0658, -0.8269,  0.0303]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:77,lr:0.005,loss:269.5889892578125
[tensor([[-0.0164,  0.3120, -0.0130],
        [ 0.0630, -0.7904,  0.0290],
        [-0.0167,  0.3123, -0.0130],
        ...,
        [-0.0157,  0.3160, -0.0134],
        [ 0.0648, -0.8171,  0.0300],
        [ 0.0658, -0.8266,  0.0303]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:78,lr:0.005,loss:268.2781677246094
[tensor([[-0.0164,  0.3118, -0.0130],
  

[tensor([[-0.0171,  0.3052, -0.0126],
        [ 0.0648, -0.7762,  0.0279],
        [-0.0174,  0.3055, -0.0125],
        ...,
        [-0.0164,  0.3092, -0.0129],
        [ 0.0666, -0.8035,  0.0290],
        [ 0.0676, -0.8133,  0.0293]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:121,lr:0.005,loss:212.86534118652344
[tensor([[-0.0171,  0.3051, -0.0125],
        [ 0.0648, -0.7758,  0.0279],
        [-0.0174,  0.3054, -0.0125],
        ...,
        [-0.0164,  0.3090, -0.0129],
        [ 0.0667, -0.8032,  0.0290],
        [ 0.0677, -0.8130,  0.0293]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:122,lr:0.005,loss:211.59878540039062
[tensor([[-0.0171,  0.3049, -0.0125],
        [ 0.0649, -0.7755,  0.0279],
        [-0.0174,  0.3052, -0.0125],
        ...,
        [-0.0164,  0.3089, -0.0129],
        [ 0.0667, -0.8029,  0.0289],
        [ 0.0677, -0.8127,  0.0293]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:123,lr:0.005,loss:210.3332061767578
[tensor([[-0.0172,  0.3047, -0.0125

epoch:162,lr:0.005,loss:161.75669860839844
[tensor([[-0.0178,  0.2988, -0.0121],
        [ 0.0665, -0.7621,  0.0269],
        [-0.0181,  0.2991, -0.0121],
        ...,
        [-0.0170,  0.3027, -0.0125],
        [ 0.0684, -0.7900,  0.0280],
        [ 0.0694, -0.8000,  0.0283]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:163,lr:0.005,loss:160.53114318847656
[tensor([[-0.0178,  0.2986, -0.0121],
        [ 0.0665, -0.7617,  0.0268],
        [-0.0181,  0.2989, -0.0121],
        ...,
        [-0.0171,  0.3026, -0.0124],
        [ 0.0684, -0.7897,  0.0279],
        [ 0.0695, -0.7997,  0.0283]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:164,lr:0.005,loss:159.3065948486328
[tensor([[-0.0178,  0.2985, -0.0121],
        [ 0.0666, -0.7614,  0.0268],
        [-0.0181,  0.2988, -0.0120],
        ...,
        [-0.0171,  0.3024, -0.0124],
        [ 0.0685, -0.7893,  0.0279],
        [ 0.0695, -0.7994,  0.0282]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:165,lr:0.005,loss:158.0830383

...

[tensor([[-0.1378,  0.0752,  0.2027],
        [ 0.4152, -0.1295, -0.5989],
        [-0.1429,  0.0450,  0.2062],
        ...,
        [-0.1441,  0.0263,  0.2057],
        [ 0.4419, -0.0490, -0.6270],
        [ 0.4514, -0.0279, -0.6379]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2787,lr:0.005,loss:0.8706803917884827
[tensor([[ 0.1378, -0.0752, -0.2027],
        [-0.4152,  0.1295,  0.5989],
        [ 0.1429, -0.0450, -0.2062],
        ...,
        [ 0.1441, -0.0263, -0.2057],
        [-0.4419,  0.0490,  0.6270],
        [-0.4514,  0.0279,  0.6379]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2788,lr:0.005,loss:0.22335997223854065
[tensor([[-0.1378,  0.0752,  0.2027],
        [ 0.4152, -0.1295, -0.5989],
        [-0.1429,  0.0450,  0.2062],
        ...,
        [-0.1441,  0.0263,  0.2057],
        [ 0.4419, -0.0490, -0.6270],
        [ 0.4514, -0.0279, -0.6379]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2789,lr:0.005,loss:0.8706819415092468
[tensor([[ 0.1378, -0.0752, -0

[tensor([[ 0.1377, -0.0753, -0.2027],
        [-0.4149,  0.1294,  0.5991],
        [ 0.1428, -0.0450, -0.2062],
        ...,
        [ 0.1440, -0.0263, -0.2057],
        [-0.4417,  0.0491,  0.6272],
        [-0.4512,  0.0279,  0.6380]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2832,lr:0.005,loss:0.22336404025554657
[tensor([[-0.1378,  0.0753,  0.2027],
        [ 0.4149, -0.1295, -0.5991],
        [-0.1428,  0.0450,  0.2062],
        ...,
        [-0.1440,  0.0263,  0.2057],
        [ 0.4417, -0.0491, -0.6272],
        [ 0.4512, -0.0279, -0.6380]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2833,lr:0.005,loss:0.8706774115562439
[tensor([[ 0.1377, -0.0753, -0.2027],
        [-0.4149,  0.1294,  0.5991],
        [ 0.1428, -0.0450, -0.2062],
        ...,
        [ 0.1440, -0.0263, -0.2057],
        [-0.4417,  0.0491,  0.6272],
        [-0.4512,  0.0279,  0.6380]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2834,lr:0.005,loss:0.2233639359474182
[tensor([[-0.1377,  0.0753,  0

[tensor([[-0.1377,  0.0753,  0.2028],
        [ 0.4148, -0.1294, -0.5992],
        [-0.1427,  0.0450,  0.2063],
        ...,
        [-0.1440,  0.0263,  0.2058],
        [ 0.4415, -0.0491, -0.6273],
        [ 0.4510, -0.0279, -0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2873,lr:0.005,loss:0.8706724047660828
[tensor([[ 0.1377, -0.0753, -0.2027],
        [-0.4148,  0.1294,  0.5992],
        [ 0.1427, -0.0450, -0.2063],
        ...,
        [ 0.1440, -0.0263, -0.2058],
        [-0.4415,  0.0491,  0.6273],
        [-0.4510,  0.0280,  0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2874,lr:0.005,loss:0.22337205708026886
[tensor([[-0.1377,  0.0753,  0.2028],
        [ 0.4148, -0.1294, -0.5992],
        [-0.1427,  0.0450,  0.2063],
        ...,
        [-0.1440,  0.0263,  0.2058],
        [ 0.4415, -0.0491, -0.6273],
        [ 0.4510, -0.0279, -0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2875,lr:0.005,loss:0.8706750869750977
[tensor([[ 0.1377, -0.0753, -0

[tensor([[ 0.1376, -0.0753, -0.2028],
        [-0.4147,  0.1294,  0.5993],
        [ 0.1427, -0.0450, -0.2063],
        ...,
        [ 0.1439, -0.0263, -0.2058],
        [-0.4414,  0.0491,  0.6274],
        [-0.4509,  0.0280,  0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2916,lr:0.005,loss:0.22338028252124786
[tensor([[-0.1377,  0.0753,  0.2028],
        [ 0.4147, -0.1295, -0.5993],
        [-0.1427,  0.0450,  0.2063],
        ...,
        [-0.1439,  0.0263,  0.2058],
        [ 0.4414, -0.0491, -0.6274],
        [ 0.4509, -0.0280, -0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2917,lr:0.005,loss:0.8706604838371277
[tensor([[ 0.1376, -0.0753, -0.2028],
        [-0.4147,  0.1294,  0.5993],
        [ 0.1427, -0.0450, -0.2063],
        ...,
        [ 0.1439, -0.0263, -0.2058],
        [-0.4414,  0.0491,  0.6274],
        [-0.4509,  0.0280,  0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2918,lr:0.005,loss:0.2233806848526001
[tensor([[-0.1377,  0.0753,  0

[tensor([[-0.1377,  0.0753,  0.2028],
        [ 0.4147, -0.1295, -0.5993],
        [-0.1427,  0.0450,  0.2063],
        ...,
        [-0.1439,  0.0263,  0.2058],
        [ 0.4414, -0.0491, -0.6274],
        [ 0.4509, -0.0280, -0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2959,lr:0.005,loss:0.8706466555595398
[tensor([[ 0.1376, -0.0753, -0.2028],
        [-0.4146,  0.1294,  0.5993],
        [ 0.1427, -0.0450, -0.2063],
        ...,
        [ 0.1439, -0.0263, -0.2058],
        [-0.4414,  0.0491,  0.6274],
        [-0.4509,  0.0280,  0.6383]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2960,lr:0.005,loss:0.2233949899673462
[tensor([[-0.1377,  0.0753,  0.2028],
        [ 0.4146, -0.1294, -0.5993],
        [-0.1427,  0.0450,  0.2063],
        ...,
        [-0.1439,  0.0263,  0.2058],
        [ 0.4414, -0.0491, -0.6274],
        [ 0.4509, -0.0280, -0.6382]], device='cuda:0', grad_fn=<AddBackward0>)]
epoch:2961,lr:0.005,loss:0.870647132396698
[tensor([[ 0.1376, -0.0753, -0.2

In [127]:
epsilon= 0.6
data_p = torch.load('data_p_gaussian_{}.pt'.format(epsilon))
target_p = torch.load('target_p_gaussian_{}.pt'.format(epsilon))
#print(data_p)

In [128]:
print(data_p.size())

torch.Size([402, 3])


In [136]:
X_all = torch.cat((X_train.to(device),data_p),0).to('cpu')
y_all = torch.cat((y_train.to(device),target_p),0).to('cpu')
print(X_all[:,0].size())

torch.Size([1072])


In [145]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linear.weight tensor([[-0.0346, -0.2471,  0.0193]], device='cuda:0')
linear.bias tensor([1.2288], device='cuda:0')


In [138]:
train_loader_all = DataLoader(LinearDataset(X_all, y_all), batch_size=batch_size_train, shuffle=False)
device = 'cuda'
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [139]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        criterion = torch.nn.BCELoss()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #output = model(data)
        loss = criterion(torch.squeeze(output), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


In [140]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            criterion = torch.nn.BCELoss()
            data, target = data.to(device), target.to(device)
            #output = model(data.view(data.size(0), -1))
            output = torch.squeeze(model(data))
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = torch.squeeze(output).round()  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [141]:
model1 = LogisticRegression(input_dim,output_dim).to(device)
optimizer1 = torch.optim.SGD(model1.parameters(), lr=0.1)

for epoch in range(100):
    train(model1, device, train_loader_all,optimizer1,epoch)
    test(model1,device, test_loader)

2it [00:00, 63.11it/s]


Train Epoch: 0 [0/1072 (0%)]	Loss: 0.729404

Test set: Average loss: 0.0021, Accuracy: 178/330 (53.94%)



2it [00:00, 51.51it/s]


Train Epoch: 1 [0/1072 (0%)]	Loss: 0.700441

Test set: Average loss: 0.0020, Accuracy: 195/330 (59.09%)



2it [00:00, 69.43it/s]


...

2it [00:00, 71.25it/s]

Train Epoch: 98 [0/1072 (0%)]	Loss: 0.144779



Test set: Average loss: 0.0004, Accuracy: 326/330 (98.79%)



2it [00:00, 66.29it/s]

Train Epoch: 99 [0/1072 (0%)]	Loss: 0.143971

Test set: Average loss: 0.0004, Accuracy: 326/330 (98.79%)



In [142]:
for name, param in model1.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linear.weight tensor([[ 2.2254, -0.0598,  0.0259]], device='cuda:0')
linear.bias tensor([-0.4209], device='cuda:0')
